<a href="https://colab.research.google.com/github/navneetslovely/QML_genom/blob/main/QNN_pauli_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom
!pip install qiskit-machine-learning
!pip install qiskit[visualization]
!pip install genomic-benchmarks
!pip install tensorflow>=2.6.0
!pip install tensorflow-addons
!pip install typing-extensions --upgrade
!pip install qiskit-aer

Mounted at /content/drive
/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 12.5 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512619 sha256=c338e44e943544f128f89fbd429e94d8fa5a6f82a341f

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import sys
import tensorflow as tf
import warnings
import numpy as np

from pathlib import Path
import os
import sys
import tensorflow as tf
import warnings
import numpy as np
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model
import matplotlib.pyplot as plt
import time
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms.classifiers import VQC
from functools import partial
# from qiskit import Aer
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.primitives import Sampler

from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import PegasosQSVC
import json
import time
import warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import Statevector
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes,ZFeatureMap, PauliFeatureMap
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_machine_learning.neural_networks import SamplerQNN
from sklearn.decomposition import PCA
algorithm_globals.random_seed = 42

from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import warnings
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import Statevector
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from functools import partial
from qiskit.circuit.library import ZZFeatureMap,ZFeatureMap
from qiskit_machine_learning.circuit.library import QNNCircuit

In [ ]:
# Load the preprocessed data
train_sequences = np.load('train_sequences.npy')
# print('train_sequences',train_sequences[:2])
train_labels = np.load('train_labels.npy')
# print('train_labels',train_labels[:2])
test_sequences = np.load('test_sequences.npy')
# print('test_sequences',test_sequences[:2])
test_labels = np.load('test_labels.npy')
# print('test_labels',test_labels[:2])
# Perform PCA transformation on the data
pca = PCA(n_components=4)
train_sequences = pca.fit_transform(train_sequences)
test_sequences= pca.transform(test_sequences)

# Split the train_sequences, train_labels, test_sequences, and test_labels into 15 parts each
split_train_sequences = np.array_split(train_sequences, 15)
split_train_labels = np.array_split(train_labels, 15)
split_test_sequences = np.array_split(test_sequences, 15)
split_test_labels = np.array_split(test_labels, 15)

# Create directories to save the parts
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_labels_parts', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_sequences_parts', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_labels_parts', exist_ok=True)

# Save each part into a separate file
for i in range(15):
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts/train_seq_{i+1}.npy', split_train_sequences[i])
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_labels_parts/train_labels_{i+1}.npy', split_train_labels[i])
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_sequences_parts/test_seq_{i+1}.npy', split_test_sequences[i])
    np.save(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/test_labels_parts/test_labels_{i+1}.npy', split_test_labels[i])

In [ ]:
feature_dim = len(train_sequences[0])
prep = PauliFeatureMap(feature_dim, reps=2)
ansatz = RealAmplitudes(num_qubits=feature_dim, reps=4)

qc= QNNCircuit(num_qubits=feature_dim ,ansatz = ansatz, feature_map=prep)
itr=0
def training_callback(weights, obj_func_eval):
        global itr
        itr += 1
        print(f"{itr} {obj_func_eval}", end=' | ')
def parity(x):
    return "{:b}".format(x).count("1") % 2
output_shape = 2

sampler_qnn = SamplerQNN(
    circuit = qc,
    interpret=parity,
    output_shape=output_shape,
    # input_gradients= True
)

sampler_classifier = NeuralNetworkClassifier(
    neural_network=sampler_qnn, optimizer=COBYLA(maxiter=40), loss= 'squared_error',callback=training_callback
)

In [ ]:

batch_size = 1024  # Specify the batch size
total_time = 0

# Placeholder for saving scores
train_scores = []
test_scores = []

In [ ]:
import pickle
#  Specify the number of epochs
num_epochs = 1
for epoch in range(num_epochs):
    start_time = time.time()  # Start time for the epoch
    epoch_train_scores = []  # Store training scores for each epoch

    # Loop over each data split
    for a in range(1, 5):
        # Load the split data
        train_sequences_part = np.load(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_sequences_parts/train_seq_{a}.npy')
        train_labels_part = np.load(f'/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QML_genom_seq/train_labels_parts/train_labels_{a}.npy')

        print(f"Training on part {a} started")

        # Perform training for each batch
        for i in range(0, len(train_sequences_part), batch_size):
            batch_sequences = train_sequences_part[i:i+batch_size]
            batch_labels = train_labels_part[i:i+batch_size]

            # Fit the model on the batch
            sampler_classifier.fit(batch_sequences, batch_labels)

            # # Optionally evaluate on training data
            # score = sampler_classifier.score(batch_sequences, batch_labels)
            # epoch_train_scores.append(score)

            print(f"Epoch {epoch + 1}, Part {a} - Batch {i // batch_size + 1} completed")

        # Optionally, save the model after each part is processed within an epoch
        # sampler_classifier.save(f'QNN_after_part_{a}_epoch_{epoch+1}')
        # print(f"Model saved after part {a}, epoch {epoch+1} completed")
        with open(f'QNN_PauliFeatureMap_after_part_{a}_epoch_{epoch+1}.pkl', 'wb') as f:
            pickle.dump(sampler_classifier, f)
        print(f"Pickle Model saved after part {a}, epoch {epoch+1} completed")


    # Average training score for the epoch
    # average_train_score = np.mean(epoch_train_scores)
    # train_scores.append(average_train_score)

    # # Evaluate on the entire test dataset after each epoch
    # test_score = sampler_classifier.score(test_sequences[:4000], test_labels[:4000])
    # test_scores.append(test_score)

    # print(f"Epoch {epoch+1} - Training Score: {average_train_score:.4f}, Test Score: {test_score:.4f}")

    # sampler_classifier.save(f'QNN_model_ZFeatureMap_epoch_{epoch+1}')
    # print(f"Model saved after epoch {epoch+1} completed")
    with open(f'QNN_PauliFeatureMap_epoch_{epoch+1}.pkl', 'wb') as f:
        pickle.dump(sampler_classifier, f)
    print(f"Model saved after pickle epoch {epoch+1} completed")

    # Calculate the time for the epoch
    end_time = time.time()
    epoch_time = end_time - start_time
    total_time += epoch_time

    print(f"Epoch {epoch+1} - Time taken: {epoch_time:.2f} seconds")

print(f"Total training time: {total_time:.2f} seconds")

# # Save scores to a file for later use
# np.save('QNN_train_scores_ZZFeatureMap.npy', np.array(train_scores))
# np.save('QNN_test_scores_ZZFeatureMap.npy', np.array(test_scores))

# train_scores_file = 'QNN_train_scores_ZFeatureMap.npy'
# test_scores_file = 'QNN_test_scores_ZFeatureMap.npy'

# if os.path.exists(train_scores_file):
#     existing_train_scores = np.load(train_scores_file)
#     updated_train_scores = np.concatenate((existing_train_scores, train_scores))
#     np.save(train_scores_file, updated_train_scores)
# else:
#     np.save(train_scores_file, np.array(train_scores))

# if os.path.exists(test_scores_file):
#     existing_test_scores = np.load(test_scores_file)
#     updated_test_scores = np.concatenate((existing_test_scores, test_scores))
#     np.save(test_scores_file, updated_test_scores)
# else:
#     np.save(test_scores_file, np.array(test_scores))

Training on part 1 started
1 0.497201136363153 | 2 0.49605645301310164 | 3 0.4990511426566067 | 4 0.500458556006453 | 5 0.4994649362255438 | 6 0.49606508351488793 | 7 0.49807046680571176 | 8 0.49812681414985255 | 9 0.49881856741403396 | 10 0.4956420616021333 | 11 0.49609315322656483 | 12 0.5000165345587542 | 13 0.4950824134067754 | 14 0.4953688676232636 | 15 0.4997339042820039 | 16 0.49477224776931517 | 17 0.49727383989453805 | 18 0.49532043611150567 | 19 0.4974589377242553 | 20 0.500130331795126 | 21 0.49752212424857484 | 22 0.48898518942159397 | 23 0.49208827115388976 | 24 0.49015312096075436 | 25 0.49073550609509264 | 26 0.4907863672612938 | 27 0.49166189114791115 | 28 0.4896569287482988 | 29 0.49290767056914264 | 30 0.4906340106748084 | 31 0.495057633775828 | 32 0.49021617157480163 | 33 0.4885093415811045 | 34 0.4907512928795831 | 35 0.48807738671993983 | 36 0.48745826415183535 | 37 0.4875594407602125 | 38 0.48857238864409147 | 39 0.4879108192176515 | 40 0.48932409376898045 | Epoch

In [ ]:

import pickle

# Replace 'your_model_filename.pkl' with the actual file name if it's different
path = "/content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom/QNN_PauliFeatureMap_epoch_1.pkl"
# /content/drive/MyDrive/Quantum_Studies_phd/PhD/QML_Genom
# Check if the file exists
if os.path.exists(path):
    # Load the model from the specified path
    with open(path, 'rb') as file:
        sampler_classifier = pickle.load(file)
    print("Model loaded successfully.")
else:
    print(f"File not found at path: {path}")

# You can now use loaded_model to make predictions or evaluate its performance

Model loaded successfully.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
predictions = sampler_classifier.predict(test_sequences[:4000])
true_labels = test_labels[:4000]

# Calculate Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Calculate Precision and Recall
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Calculate F1-Score
f1 = f1_score(true_labels, predictions)
print(f"F1 Score: {f1:.4f}")

# Calculate ROC-AUC Score
# Since it's a binary classification, directly pass the true labels and predictions
roc_auc = roc_auc_score(true_labels, predictions)
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Generate a full classification report
report = classification_report(true_labels, predictions)
print("Classification Report:\n", report)

Accuracy: 0.5052
Precision: 0.5167
Recall: 0.4775
F1 Score: 0.4963
ROC-AUC Score: 0.5058
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.53      0.51      1958
           1       0.52      0.48      0.50      2042

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.50      4000

